In [1]:
from whoosh.index import open_dir
from whoosh.qparser import MultifieldParser, QueryParser, FuzzyTermPlugin
from whoosh import scoring, sorting
from whoosh.query import Term
from whoosh.reading import IndexReader
from recommend import MyRecommender

In [2]:
ix = open_dir('index/whoosh_index')

In [65]:
with ix.searcher(weighting=scoring.BM25F()) as searcher:
    query_parser = MultifieldParser(['filtered_title', 'pure_text', 'anchor_backward_text', 'anchor_forward_text'], schema=ix.schema)
    query = query_parser.parse('实验')
    results = searcher.search(query, limit=None)
    print(len(results))

968


In [27]:
with ix.searcher(weighting=scoring.BM25F()) as searcher:
    print(list(searcher.lexicon('url')))

[b'http://aiguo.nankai.edu.cn/', b'http://aiguo.nankai.edu.cn/main.htm', b'http://binhai.nankai.edu.cn/', b'http://binhai.nankai.edu.cn/djzs/', b'http://binhai.nankai.edu.cn/fwzn.htm', b'http://binhai.nankai.edu.cn/gongk/', b'http://binhai.nankai.edu.cn/hjgc/', b'http://binhai.nankai.edu.cn/index.htm', b'http://binhai.nankai.edu.cn/international/hwfx.htm', b'http://binhai.nankai.edu.cn/international/hzjl.htm', b'http://binhai.nankai.edu.cn/international/index.htm', b'http://binhai.nankai.edu.cn/international/xwzx.htm', b'http://binhai.nankai.edu.cn/jwb/', b'http://binhai.nankai.edu.cn/rcsz/szjs.htm', b'http://binhai.nankai.edu.cn/xww/index.htm', b'http://binhai.nankai.edu.cn/zs/', b'http://binhai.nankai.edu.cn/zs/bkzn/zsjh.htm', b'http://booking.stat.nankai.edu.cn', b'http://cfc.nankai.edu.cn/', b'http://en.cim.nankai.edu.cn', b'http://en.cim.nankai.edu.cn/a110th_Anniv__of_Chern.htm', b'http://en.economics.nankai.edu.cn', b'http://en.fcollege.nankai.edu.cn/', b'http://en.finance.nankai

In [20]:
query_parser = QueryParser('filtered_title', schema=ix.schema)
query = query_parser.parse('南*大学')
print(query)
with ix.searcher() as searcher:
    results = searcher.search(query, limit=None)
    for result in results:
        print(result['filtered_title'])

(filtered_title:南 AND filtered_title:大学)
津,南,开,专题报道,南开大学
南大学生回校,专题报道,南开大学
南大楼上看火车,南开故事,南开大学
南大学生被阻沧州,专题报道,南开大学
今晚报,习近平给南大学生回信,媒体南开,南开大学
天津电视台,百年南开心盛世梨园情,媒体南开,南开大学
物理之美科学力量,系列科普课程走进南大附小,综合新闻,南开大学
今晚报,南大生态文明研究院今成立,媒体南开,南开大学
汪磊,深切悼念南开环科创始人戴树桂教授,专题报道,南开大学
今晚报,南大百年纪念邮票明发行,媒体南开,南开大学
今晚报,南大百年校庆纪念币,日发行,媒体南开,南开大学
天津日报,南大学子放弃家乡高薪岗位,奔赴西部圆梦,媒体南开,南开大学
今晚报,南大图书馆百年馆庆展开放,媒体南开,南开大学
今晚报,南大建校,周年金银纪念币昨发行,媒体南开,南开大学
爱国立行记录南开战,疫,十二时辰,主题党日活动,专题报道,南开大学
知行南开,南开赴甘肃灵台实践团走乡入镇开展调研,综合新闻,南开大学
喜迎十九大奋进看南开,公能日新强党建百年南开创一流,新闻聚焦,南开大学
天津工人报,南大图书馆举办百年馆庆系列展览,媒体南开,南开大学
经济学院党委,南陈北李相约建党民族复兴青年担当,专题报道,南开大学
天津日报,市领导察看南大天大新校区开学前准备工作,媒体南开,南开大学
天津日报,南大首发就业指导陪准毕业生,个月,媒体南开,南开大学
每日新报,著名教育家,化学家,中科院院士,南大教授申泮文昨逝世,媒体南开,南开大学


In [13]:
query_parser = QueryParser('url', schema=ix.schema)
query = query_parser.parse('*jwc.nankai.edu.cn*')
with ix.searcher() as searcher:
    results = searcher.search(query, limit=None)
    print(len(results))
    for result in results:
        print(result['url'])

3
http://jwc.nankai.edu.cn/
http://jwc.nankai.edu.cn/2022/0510/c20a449384/page.htm
http://jwc.nankai.edu.cn/a4/99/c5098a42137/page.psp


In [77]:
with ix.searcher() as searcher:
    user_q = QueryParser('filtered_title', schema=ix.schema).parse('南开大学')
    allow_q = Term('filtered_title', '南开大学')
    deny_q = Term('filtered_title', '南开')
    results = searcher.search(user_q, filter=deny_q, limit=None, terms=True)
    if results.has_matched_terms():
        print(results.matched_terms())
        for match in results.matched_terms():
            print(match[0], str(match[1], encoding='utf-8'))
    else:
        print('not matched')

{('filtered_title', b'\xe5\x8d\x97\xe5\xbc\x80'), ('filtered_title', b'\xe5\xa4\xa7\xe5\xad\xa6'), ('filtered_title', b'\xe5\xbc\x80\xe5\xa4\xa7'), ('filtered_title', b'\xe5\x8d\x97\xe5\xbc\x80\xe5\xa4\xa7\xe5\xad\xa6')}
filtered_title 南开
filtered_title 大学
filtered_title 开大
filtered_title 南开大学


In [9]:
user_query = QueryParser('filtered_title', schema=ix.schema).parse('信息公开')
term_set = user_query.existing_terms(ix.reader())
for term in term_set:
    print(term[0], str(term[1], encoding='utf-8'))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.540 seconds.
Prefix dict has been built successfully.


filtered_title 公开
filtered_title 信息


In [16]:
query_parser = QueryParser('filtered_title', schema=ix.schema)
query_parser.add_plugin(FuzzyTermPlugin())
query = query_parser.parse('信心~')
print(query)
with ix.searcher() as searcher:
    results = searcher.search(query, limit=None)
    print(len(results))
    for result in results:
        print(result['url'])

filtered_title:信心~
534
http://xxgk.nankai.edu.cn/
http://news.nankai.edu.cn/zhxw/system/2021/12/21/030049632.shtml
http://news.nankai.edu.cn/ywsd/system/2021/04/23/030045629.shtml
http://binhai.nankai.edu.cn/zs/bkzn/zsjh.htm
http://binhai.nankai.edu.cn/gongk/
https://nkjd.nankai.edu.cn/
http://binhai.nankai.edu.cn/international/xwzx.htm
http://binhai.nankai.edu.cn/zs/
https://fy.nankai.edu.cn/8654/list.htm
https://fy.nankai.edu.cn/8658/list.htm
https://xxb.nankai.edu.cn/
https://cyber.nankai.edu.cn/13350/list.htm
https://cyber.nankai.edu.cn/13345/list.htm
https://cc.nankai.edu.cn/teachers/recruitmentInfo/main.psp
http://ndst.nankai.edu.cn/23635/list.htm
https://yzb.nankai.edu.cn/2022/0623/c5508a459343/page.htm
https://cc.nankai.edu.cn/13299/list.htm
https://cc.nankai.edu.cn/13294/list.htm
http://cfc.nankai.edu.cn/
http://www.cim.nankai.edu.cn/6707/hy050810.psp
https://ceo.nankai.edu.cn/
https://eilab.nankai.edu.cn/
https://imo.nankai.edu.cn/info/1054/1092.htm
https://ceo.nankai.edu.cn/

In [12]:
user_query = QueryParser('filtered_title', schema=ix.schema).parse('\"本科质量\"')
with ix.searcher() as searcher:
    results = searcher.search(user_query, limit=None)
    print(len(results))
    for result in results:
        print(result['filtered_title'])

0


In [21]:
import hashlib

def string_hash(string):
    return hashlib.md5(string.encode('utf-8')).hexdigest()

In [26]:
print(string_hash('南开大学'))
print(string_hash(''))

968b58913d5ed37edf3b12da6906a704
d41d8cd98f00b204e9800998ecf8427e


In [2]:
recommender = MyRecommender(user_id=1)
recommender.recommend()

[('http://jwc.nankai.edu.cn/', 7.323647594543401),
 ('http://news.nankai.edu.cn/mtnk/system/2016/11/17/000305979.shtml?wd=%CE%E2%D6%BE%B3%C9+%D0%C2%CE%C5%C1%AA%B2%A5&x=13&y=10',
  7.323647594543401),
 ('http://news.nankai.edu.cn/ztbd/system/2021/06/27/030047036.shtml',
  6.723053019297644),
 ('https://yzb.nankai.edu.cn/2022/0623/c5508a459438/page.htm',
  5.721139227572456),
 ('http://news.nankai.edu.cn/ywsd/system/2022/02/23/030050367.shtml',
  2.887581946738247),
 ('http://less.nankai.edu.cn/equipment/content/8', 1.7888543819998317),
 ('http://energy.nankai.edu.cn/article/725', 0.8749565038354035),
 ('https://teda.nankai.edu.cn/main.htm', 0.873477180721818),
 ('http://news.nankai.edu.cn/ywsd/system/2017/07/12/000338495.shtml',
  0.7678252778262218),
 ('http://news.nankai.edu.cn/ywsd/system/2015/07/22/000243055.shtml',
  0.7071067811865476),
 ('http://news.nankai.edu.cn/ywsd/system/2021/06/09/030046563.shtml',
  0.7071067811865476),
 ('http://news.nankai.edu.cn/mtnk/system/2017/06/22/0